This notebook originates from an article that seems like a simple replacement piece, but in reality, there are many pitfalls to be aware of, especially if you're using Azure OpenAI instead of OpenAI. For more detail: https://towardsdatascience.com/improving-retrieval-performance-in-rag-pipelines-with-hybrid-search-c75203c2f2f5#6b8c .

In [1]:
import dotenv
dotenv.load_dotenv('../../.env')

True

In [2]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [3]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

#### Key parts that need to be adapted

**I must say, the integration of Weaviate's hybrid search client with Azure OpenAI is designed very counterintuitively**, And as always, if you have GPT-4, then please discard GPT-3.5.

In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
import os

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-Azure-Api-Key": os.getenv("AZURE_EMBED_API_KEY"),
    },
)

class_obj = {
    "class": "LangChain",
    "vectorizer": "text2vec-openai",
    "moduleConfig": {
        "text2vec-openai": { # One of the most foolish API designs I've ever seen. Why we need resourceName and deploymentId here?
            "baseURL": os.getenv("AZURE_EMBED_ENDPOINT"),
            "resourceName": os.getenv("AZURE_EMBED_RESOURCE_NAME"),
            "deploymentId": "text-embedding-ada-002",
        }
    },
}
client.schema.delete_all()
client.schema.create_class(class_obj)

from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

retriever = WeaviateHybridSearchRetriever(
    alpha=0.5,  # defaults to 0.5, which is equal weighting between keyword and semantic search
    client=client,  # keyword arguments to pass to the Weaviate client
    index_name="LangChain",  # The name of the index to use
    text_key="text",  # The name of the text key to use
    attributes=[],  # The attributes to return in the results
)

retriever.add_documents(chunks)

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
llm = AzureChatOpenAI(azure_deployment="gpt-4", openai_api_version="2023-08-01-preview", temperature=0) # If you're using GPT-3.5 here, then the results are very unpredictable.
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

'The president honored Justice Stephen Breyer for his service as an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. He thanked Justice Breyer for his service to the country. The president also mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer’s legacy of excellence.'